<a href="https://colab.research.google.com/github/rklepov/OTUS-ClickHouse-2025-03/blob/main/2025-04-21_Functions/OTUS_regular_agg_ud_functions_for_students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Настройка ClickHouse

In [ ]:
# Установка ClickHouse
# Во время загрузки потребует ввести ПАРОЛЬ!
# Если в конце будет Ok! значит установка завершена
!sudo apt-get update
!sudo apt-get install curl gnupg2 apt-transport-https ca-certificates
!sudo sh -c 'mkdir -p /usr/share/keyrings && curl -s https://builds.altinity.cloud/apt-repo/pubkey.gpg | gpg --dearmor > /usr/share/keyrings/altinity-archive-keyring.gpg'
!sudo sh -c 'echo "deb [signed-by=/usr/share/keyrings/altinity-archive-keyring.gpg] https://builds.altinity.cloud/apt-repo stable main" > /etc/apt/sources.list.d/altinity-dev.list'
!sudo apt-get update
#!sudo apt-get install clickhouse-common-static=21.8.10.1.altinitystable clickhouse-client=21.8.10.1.altinitystable clickhouse-server=21.8.10.1.altinitystable
!sudo apt-get install clickhouse-common-static=25.3.2.39.altinitystable clickhouse-client=25.3.2.39.altinitystable clickhouse-server=25.3.2.39.altinitystable
!sudo apt-get install clickhouse-client clickhouse-server

!sudo service clickhouse-server start
!sudo service clickhouse-server status
!curl localhost:8123

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [75.2 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,604 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/mai

In [ ]:
# Если хочется поработать с ClickHouse не через Colab
# Регаемся на https://ngrok.com/
# ngrok — это сервис, который позволяет открыть доступ к внутренним ресурсам машины,
# на которой он запущен, из внешней сети, путем создания публичного адреса,
# все запросы на который будут переброшены на локальный адрес и заданный порт.

!pip install pyngrok

# токен можно найти тут https://dashboard.ngrok.com/get-started/setup
!ngrok authtoken 2kNHm66ePlrYxnCzGLuUTBMCel6_7NCxMyaffUuW99134h1VY

# Эта команда отображет веб морду на другой адрес
# Адрес можно посмотреть тут https://dashboard.ngrok.com/cloud-edge/endpoints
!nohup ngrok http 8123 > /dev/null &

# Не забудьте в адресную строку дописать /play, чтобы видеть не просто "Ок.", а приложение, в которое можно писать запросы
# должно получится что-то вида https://eef0-34-74-98-209.ngrok-free.app/play

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
nohup: redirecting stderr to stdout


In [ ]:
# Работать будем через TCP-протокол, который реализован через библиотеку clickhouse-driver
!pip install clickhouse-driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.5 MB/s eta 0:00:00


In [ ]:
from clickhouse_driver import Client

client = Client(host='localhost', user='default', password='12345')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# REGULAR FUNCTIONS

[Типы данных в ClickHouse](https://clickhouse.com/docs/en/sql-reference/data-types)

### Conditional

In [ ]:
# if(cond, then, else)
client.execute("SELECT if(False, 2+2, 2+6)")

[(8,)]

In [ ]:
client.execute( '''
select
   1 as num,
   case
     when num = 3 then 'third'
     when num = 2 then 'second'
     when num = 1 then 'first'
     else 'zero'
   end
''')

client.execute('''
select
   10 as num,
   multiIf(num=1, 'first', num=2, 'second', 'third')
   ''')

[(10, 'third')]

In [ ]:
# Движок LOG Движки разработаны для сценариев,
# когда необходимо быстро записывать много таблиц с небольшим объёмом данных
# (менее 1 миллиона строк), а затем читать их целиком.
# используем как временную таблицу или как промежуточную
client.execute('''
CREATE TABLE table1
(
    id UInt64,
    name String,
    date DateTime,
    score UInt64
)
ENGINE = Log
''')

client.execute('''
INSERT INTO table1 (*) VALUES (0, 'Max', '2020-01-01', 50),
(1, 'Dan', '2020-01-01', 12),
(2, 'Alex', '2020-01-01', 95),
(3, 'Alex', '2020-01-02', 12),
(4, 'Max', '2020-01-02', 66),
(5, 'Dan', '2020-01-02', 69),
(6, 'Dan', '2020-01-03', 18),
(7, 'Alex', '2020-01-03', 66),
(8, 'Max', '2020-01-03', 86)
''')

[]

In [ ]:
# alias
client.execute('''
select id as d
  , d/2
from table1
where like(name, '%A%')
''')

In [ ]:
# ilike, notLike like(haystack, pattern)
client.execute('''
select *
from table1
where like(name, '%A%')
''')

[(2, 'Alex', datetime.datetime(2020, 1, 1, 0, 0), 95),
 (3, 'Alex', datetime.datetime(2020, 1, 2, 0, 0), 72),
 (7, 'Alex', datetime.datetime(2020, 1, 3, 0, 0), 60)]

### DateTime functions

Для хранения времени в ClickHouse и во многих других базах данных и языках используется так называемый Unix timestamp — это числовое представление времени. За старт берётся 1 января 1970 года, после чего прибавляется некоторое количество секунд, это и является представлением времени. Но в основном оно показывается в виде даты и времени, к которым мы привыкли.

In [ ]:
# now([timezone])


client.execute("select * FROM system.time_zones") #список доступных таймзон

client.execute("SELECT now(), today(), now() - interval 1 day")

# toYYYYMM() Переводит дату или дату со временем в число типа UInt32, содержащее номер года и месяца (YYYY * 100 + MM).

[(datetime.datetime(2024, 10, 12, 21, 21, 12),
  datetime.date(2024, 10, 12),
  datetime.datetime(2024, 10, 11, 21, 21, 12),
  202410)]

In [ ]:
# toDate() переводит в локальное время, если не указано другое
client.execute('''
select
  toDateTime('2024-10-01 23:00:00') as dt
  , toDate(dt) as date_local
  , toDate(dt, 'Asia/Yekaterinburg') AS date_yekat
  , toString(dt, 'US/Samoa') AS time_samoa
  , parseDateTimeBestEffort(time_samoa) as parsing_dt
  , toStartOfWeek(dt, 1) as week_dt
  , formatDateTime(dt, '%Y_%m_%d') AS formatted_date
  , toDate(arrayJoin(range(toUInt32(today()), toUInt32(today()) + 30))) AS date
''')

# В ClickHouse существуют различные функции для преобразования строки в дату и время.
# Например, самая простая функция toDateTime предполагает, что дата будет записана в
# удобочитаемом виде YYYY-MM-DD HH:MM:SS к которому мы все привыкли.

# Однако для более сложных комбинаций существуют функции, такие как parseDateTimeBestEffort(),
# которая попытается подставить подходящую дату. Но и она не справится со всеми возможными форматами
# Nov 30, 2016 12:00:00 PDT

[(datetime.datetime(2024, 10, 1, 23, 0),
  datetime.date(2024, 10, 1),
  datetime.date(2024, 10, 2),
  '2024-10-01 12:00:00',
  datetime.datetime(2024, 10, 1, 12, 0),
  datetime.date(2024, 9, 30))]

In [ ]:
# Null в формате dateTime - '1970-01-01'
client.execute('''
select
  toDate('2024-01-01') as first_dt
  , toDate('2023-10-29') as second_dt
  , dateDiff('day', second_dt, first_dt) as day_diff
  , dateDiff('month', second_dt, first_dt) as month_diff
  , dateDiff('year', second_dt, first_dt) as year_diff
''')

# date_diff('unit', startdate, enddate, [timezone])

[(datetime.date(2024, 1, 1), datetime.date(2023, 10, 29), 64, 3, 1)]

### Array functions

Массивы — это ещё один тип данных, представляющий собой итерабельный список элементов. Он чем-то похож на list в языке Python. Этот инструмент весьма полезен при сложной обработке данных, а также позволяет заменять оконные функции, которые появились в ClickHouse совсем недавно. Массив может состоять только из одного типа данных. К элементам массива можно обращаться через индекс  [2]



Теперь попробуем применить к этому lambda функцию. Если вы знакомы с Python, то такая конструкция вас не удивит. Это некоторая неименованная функция. Чтобы применить её к нашему массиву, используют arrayMap (или другие аналогичные функции, например, arrayFilter).

In [ ]:
client.execute('''
SELECT [1,4,3,2] as array_1,
  array_1[1] as first,
  array_1[-1] as last,
  arraySum(array_1) as sum_array,
  reverse(array_1) as rev_array,
  length(array_1) as len,
  arrayEnumerate(array_1) as enum,
  arraySort(array_1) as sorted,
  arrayFilter(x -> x > 2, array_1) as filtered,
  arrayMap(x -> if(x<3, (x + 2), (x+9)), array_1) as mapped
''')

# arraySort([func,] arr)
# arrayMap(func, arr)

[([1, 2, 3], 1, 3, 6, [3, 2, 1], 3)]

In [ ]:
# Это очень необычная функция.
# Обычные функции не изменяют набор строк, а просто изменяют значения в каждой строке (map).
# Агрегатные функции сжимают набор строк (fold или reduce). Функция arrayJoinберет каждую строку и генерирует набор строк (unfold).
# Эта функция принимает массив в качестве аргумента и распространяет исходную строку на несколько строк для количества элементов в массиве.
# Все значения в столбцах просто копируются, за исключением значений в столбце, где эта функция применяется;
# они заменяются соответствующим значением массива.
# Сворачивание и разворачивание, антипод arrayGroup

client.execute('''
SELECT arrayJoin([1,2,3] as num) as dst,
  num
''')

[(1, [1, 2, 3]), (2, [1, 2, 3]), (3, [1, 2, 3])]

In [ ]:
#Запрос может использовать несколько arrayJoin функций. В этом случае преобразование выполняется несколько раз, а строки умножаются.

client.execute('''
select
  arrayJoin(colors) as color
  , arrayJoin(sizes) as size
from
(
  select ['red', 'green', 'black'] as colors
    , ['37', '38', '39'] as sizes
)
''')

[('red', '37'),
 ('red', '38'),
 ('red', '39'),
 ('green', '37'),
 ('green', '38'),
 ('green', '39'),
 ('black', '37'),
 ('black', '38'),
 ('black', '39')]

In [ ]:
# Обратите внимание на синтаксис ARRAY JOIN в запросе SELECT, который предоставляет более широкие возможности.
# ARRAY JOIN Позволяет преобразовывать несколько массивов с одинаковым количеством элементов одновременно.

client.execute('''
select
  size
  , centimetre
from
(
  select ['37', '38', '39'] as sizes
    , ['23', '24', '25'] as centimetres
)
ARRAY JOIN
    sizes AS size,
    centimetres AS centimetre
''')

[('37', '23'), ('38', '24'), ('39', '25')]

### JSON

На сегодняшний момент тип данных `JSON` все еще экспериментальный (релиз был в июле 2024). Подробности можно посмотреть на [GitHub](https://github.com/ClickHouse/ClickHouse/pull/66444/). Если хочется потрогать руками, то не забудьте разрешить использование экспериментальных объектов:

```
SET allow_experimental_object_type = 1;
```



In [ ]:
client.execute('''
CREATE TABLE table_json
(
    id UInt64,
    date Date,
    json String
)
ENGINE = TinyLog
''')

[]

In [ ]:
client.execute('''
INSERT INTO table_json (*) VALUES
    (1, '2020-01-01','{"name":"Max","age":30,"tags":["A", "B"],"active":true,"params":{"param1":"purchase","param2":3,"param3":[10,24,35,14,12]}}'),
    (2, '2020-01-01','{"name":"Dan","age":45,"tags":["A", "B"],"active":true,"params":{"param1":"click","param2":2,"param3":[33,68,50,82,42]}}')
''')

[]

In [ ]:
# Существует два набора функций для анализа JSON:
# - simpleJSON*( visitParam*), который предназначен для чрезвычайно быстрого анализа ограниченного подмножества JSON.
# - JSONExtract*который предназначен для разбора обычного JSON.

client.execute('''
select id,
  isValidJSON(json) as json_valid,
  simpleJSONHas(json, 'name'),
  visitParamHas(json, 'surname'),
  visitParamExtractString(json, 'name') as name,
  JSONExtractBool(json, 'active') as active,
  JSONExtractInt(json, 'age') as age,
  JSONExtractArrayRaw(
          JSONExtractString(json, 'params'),
             'param3') as param3
from table_json
''')

[(1, 1, 1, 0, 'Max', 1, 30, ['10', '24', '35', '14', '12']),
 (2, 1, 1, 0, 'Dan', 1, 45, ['33', '68', '50', '82', '42'])]

# AGGREGATE FUNCTIONS

ClickHouse поддерживает классический синтаксис операций для группировки GROUP BY, и HAVING для последующей фильтрации сгруппированных данных, а также множество агрегатных функций чтобы посчитать все на свете

In [ ]:
client.execute('''
select uniq(name), uniqExact(name)
from table1
''')

# Приближённо вычисляет количество различных значений аргумента.
# Вычисляет точное количество различных значений аргументов.
# Функция uniqExact расходует больше оперативной памяти, чем функция uniq,
# так как размер состояния неограниченно растёт по мере роста количества различных значений.

Особенностью диалекта ClickHouse является наличие комбинаторов. Очень полезный из них If. При данном дополнении агрегатная функция будет применяться только к тем значениям в строках которой будет возвращаться True. Можно это рассматривать как предварительный WHERE но все вычисления происходят для каждой функции отдельно, или аналогично CASE WHEN внутри агрегатной функции. Однако более нативный вариант для ClickHouse это использование комбинатора. Данная конструкция весьма полезна, и помогает иногда обходиться без дополнительных JOIN операторов.

In [ ]:
client.execute('''
select date,
    argMax(name, score),
    argMaxIf(name, score, mod(id,2))
from table1
group by date
''')

# argMax(arg, val)
# argMaxIf(arg, val, cond)

In [ ]:
client.execute('''
select groupArray(name)
from table1
''')

[(['Max', 'Dan', 'Alex', 'Alex', 'Max', 'Dan', 'Dan', 'Alex', 'Max'],)]

In [ ]:
client.execute('''
select arraySum(arrayMap(x -> toInt64(x), list)) as arr_sum
from (
select JSONExtractArrayRaw(
          JSONExtractString(json, 'params'),
             'param3') as list
from table_json) t1
''')

[(95,), (275,)]

Модификатор WITH ROLLUP применяется для подсчета подытогов для ключевых выражений. При этом учитывается порядок следования ключевых выражений в списке GROUP BY. Подытоги подсчитываются в обратном порядке: сначала для последнего ключевого выражения в списке, потом для предпоследнего и так далее вплоть до самого первого ключевого выражения.

Строки с подытогами добавляются в конец результирующей таблицы. В колонках, по которым строки уже сгруппированы, указывается значение 0 или пустая строка.

In [ ]:
client.execute('''
select date, name, count(1)
from table1
group by date, name with ROLLUP
''')

Модификатор WITH CUBE применяется для расчета подытогов по всем комбинациям группировки ключевых выражений в списке GROUP BY.

Строки с подытогами добавляются в конец результирующей таблицы. В колонках, по которым выполняется группировка, указывается значение 0 или пустая строка.


In [ ]:
client.execute('''
select date, name, count(1)
from table1
group by date, name with CUBE
''')

# USER DEFINED FUNCTIONS



ClickHouse может вызывать любую внешнюю исполняемую программу или скрипт для обработки данных. py, sh

### Исполняемые пользовательские функции

Файл с именем вида `*_function.xm*` кладем в `/etc/clickhouse-server/`

Файл с функцией с именем вида `*_function.py` кладем в `/var/lib/clickhouse/user_scripts/` и для всех пользователей даем права на исполнение

https://clickhouse.com/docs/en/sql-reference/functions/udf

In [ ]:
! chmod a+x /var/lib/clickhouse/user_scripts/test_function.py

In [ ]:
!clickhouse-client --user default --password 12345 --query "SELECT test_function_python(toUInt64(200.2));"

Value 200


### Пользовательские функции SQL

Создает пользовательскую функцию (UDF) из лямбда-выражения. Выражение должно состоять из параметров функции, констант, операторов или других вызовов функций.

Функция может иметь произвольное количество параметров.

Есть несколько ограничений:

Имя функции должно быть уникальным среди пользовательских и системных функций.
Рекурсивные функции не допускаются.
Все переменные, используемые функцией, должны быть указаны в списке ее параметров.
Если какое-либо ограничение нарушается, возникает исключение.

Посмотреть все данные о функциях можно в system.functions

In [ ]:
client.execute("CREATE FUNCTION linearEquation AS (x, k, b) -> k*x + b;")
client.execute("SELECT number, linearEquation(number, 2, 1) FROM numbers(3);")
# numbers(N) - возвращает таблицу с единственным столбцом number (UInt64), содержащим натуральные числа от 0 до N-1

[(0, 1), (1, 3), (2, 5)]

In [ ]:
# Просмотр всех UDF
client.execute('''
SELECT name, create_query FROM system.functions
WHERE origin = 'SQLUserDefined'
''')

[('linearEquation',
  'CREATE FUNCTION linearEquation AS (x, k, b) -> ((k * x) + b)')]

In [ ]:
client.execute("DROP FUNCTION linearEquation;")

[]

In [ ]:
client.execute("select * from system.functions where name = 'test_function_python'")

[('test_function_python',
  0,
  0,
  '',
  '',
  'ExecutableUserDefined',
  '',
  '',
  '',
  '',
  '',
  '')]